In [22]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import json

In [5]:
Directory = "UD_English-EWT/"
#Order: dev, test, train
Filename = ["en_ewt-ud-dev.conllu"]
Jsons = []

for f in Filename:
    Path = "".join((Directory, f))
    
    with open(Path, 'r', encoding='UTF-8') as j:
        i = 0
        for line in j:
            conllu = line.split()
   
    j.close()


In [11]:
def hasalpha(word):
    for x in word:
        if x.isalpha():
            return True
    return False

def hasnumeric(word):
    for x in word:
        if x.isnumeric():
            return True
    return False

def hasspecial(word):
    for x in word:
        if x.isalnum() is False:
            return True
    return False

def create_features(filepath, tagtype):
    
    other_tag = 4 if tagtype == 3 else 3
    
    with open(filepath, 'r', encoding='UTF-8') as f:
        x_toadd = []
        y_toadd = []
        x_final = []
        y_final = []
        
        for line in f:
                       
            conllu = line.split()
            #case: end of sentence reached
            if len(conllu) == 0:
                x_final.append(x_toadd)
                y_final.append(y_toadd)
                x_toadd = []
                y_toadd = []                
            elif conllu[0].isnumeric():
                word = conllu[1]
                #UPOS tag if we're predicting XPOS tag, XPOS tag if we're predicting UPOS tag
                other_tag_word = conllu[other_tag]
                other_tag_name = 'XPOS' if other_tag == 4 else 'UPOS'
                extra_tags = conllu[5].split('|')
                y_tag = conllu[tagtype]
                
                
                features = ['bias',
                            'word.lower=' + word.lower(),
                            'word.%s=%s' % (other_tag_name, other_tag_word),
                            'word.iscapitalized=%s' % word.istitle(),
                            'word.hasnumeric=%s' % hasnumeric(word),
                            'word.hasalpha=%s' % hasalpha(word),
                            'word.hasspecial=%s' % hasspecial(word)
                           ]
#                             'word.prefix=' + word[:3],
#                             'word.suffix=' + word[-3:]]
                
                if conllu[1].lower() != conllu[2].lower():
                    features.append('word.baseform=' + conllu[2].lower())
    
                for x in extra_tags:
                    if 'PronType' in x:
                        features.append('word.prontype=' + x[9:])
                    if 'Foreign' in x:
                        features.append('word.foreign=' + x[8:])
                        
                x_toadd.append(features)
                y_toadd.append(y_tag)
                
    f.close()
                
    return x_final, y_final
                

In [12]:
#3 if predicting UPOS, 4 if predicting XPOS
dev_x_unneighbored, dev_y = create_features("UD_English-EWT/en_ewt-ud-dev.conllu", 4)
test_x_unneighbored, test_y = create_features("UD_English-EWT/en_ewt-ud-test.conllu", 4)
train_x_unneighbored, train_y = create_features("UD_English-EWT/en_ewt-ud-train.conllu", 4)

<class 'int'>
<class 'int'>
<class 'int'>


In [18]:
def add_neighbors(x_data):
    
    x_data_final = []
    
    for sentence in x_data:
        final_sentence = []
        for i in range(0, len(sentence)):
            modified_word = []
            modified_word.extend(sentence[i])

            if i == 0:
                modified_word.append('BOS')
            else:
                prev_word = sentence[i-1]
                modified_word.extend(['-1:' + x for x in prev_word[1:]])

            if i == (len(sentence)-1):
                modified_word.append('EOS')
            else:
                next_word = sentence[i+1]
                modified_word.extend(['+1:' + x for x in next_word[1:]])

            final_sentence.append(modified_word)
        x_data_final.append(final_sentence)
            
    return x_data_final

In [19]:
dev_x = add_neighbors(dev_x_unneighbored)
test_x = add_neighbors(test_x_unneighbored)
train_x = add_neighbors(train_x_unneighbored)

In [20]:
def x_test():
    for i in range(0, len(dev_y)):
        for j in range(0,len(dev_y[i])):
            if dev_y[i][j] == 'X':
                print(dev_x[i][j])
                return
            
x_test()

In [23]:
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(train_x, train_y):
    trainer.append(xseq, yseq)

In [24]:
trainer.set_params({
    'c1': 1.0,   
    'c2': 1e-3,  
    'max_iterations': 50,  

    #'feature.possible_transitions': True
})

In [25]:
trainer.train('test')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 134638
Seconds required: 0.824

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 671752.177676
Feature norm: 1.000000
Error norm: 101868.231211
Active features: 55001
Line search trials: 1
Line search step: 0.000007
Seconds required for this iteration: 3.428

***** Iteration #2 *****
Loss: 527480.038968
Feature norm: 3.582148
Error norm: 113745.787968
Active features: 53497
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.688

***** Iteration #3 *****
Loss: 451148.939615
Feature norm: 5.448802
Error norm: 95410.591712
Active features: 56143
Line search trials: 1
Line search step: 1.000000
Seconds require

***** Iteration #42 *****
Loss: 25811.899847
Feature norm: 139.515093
Error norm: 910.954688
Active features: 13182
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 3.130

***** Iteration #43 *****
Loss: 25753.060083
Feature norm: 139.567090
Error norm: 621.001480
Active features: 13167
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.593

***** Iteration #44 *****
Loss: 25646.307089
Feature norm: 139.972853
Error norm: 339.354677
Active features: 13012
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.630

***** Iteration #45 *****
Loss: 25514.013691
Feature norm: 141.447733
Error norm: 836.437230
Active features: 12682
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.669

***** Iteration #46 *****
Loss: 25451.112235
Feature norm: 142.295595
Error norm: 2633.866060
Active features: 12486
Line search trials: 1
Line search step: 1.00000

In [26]:
tagger = pycrfsuite.Tagger()
tagger.open('test')

In [27]:
example_sent = test_x[1]

print("Predicted:", ' '.join(tagger.tag(example_sent)))
print("Correct:  ", ' '.join(test_y[1]))

Predicted: WP IN NNP VBD IN PRP$ NN HYPH NN -LRB- CC RB NN -RRB- NN IN DT RB HYPH JJ NN NN .
Correct:   WP IN NNP VBD IN PRP$ NN HYPH NN -LRB- CC RB NN -RRB- NNS IN DT RB HYPH JJ NN NN .


In [48]:
def evaluate(data_x, data_y):
    scores = {}
    tagger_2 = pycrfsuite.Tagger()
    tagger_2.open('test')
    
    for i in range(0, len(data_x)):
        predicted = tagger_2.tag(data_x[i])
        actual = data_y[i]
        
        for j in range(0, len(predicted)):
            
            if predicted[j] not in scores.keys():
                scores[predicted[j]] = {'TP': 0, 'FP': 0, 'FN': 0}
            if actual[j] not in scores.keys():
                scores[actual[j]] = {'TP': 0, 'FP': 0, 'FN': 0}
            
            if predicted[j] == actual[j]:
                scores[predicted[j]]['TP'] += 1
            else:
                scores[predicted[j]]['FP'] += 1
                scores[actual[j]]['FN'] += 1
        
    return scores
        
def print_evaluate(data):
    for x in data.keys():
        print("Stats for %s:" % x)
        metrics = data[x]
        
        precision = None
        if metrics['TP'] == 0 and metrics['FP'] == 0:
            precision = 0
        else:
            precision = metrics['TP']/(metrics['TP'] + metrics['FP'])
        
        print("Precision: ", precision)
        
        recall = None
        if metrics['TP'] == 0 and metrics['FN'] == 0:
            recall = 0
        else:
            recall = metrics['TP']/(metrics['TP'] + metrics['FN']) 
        print("Recall: ", recall)
        
        F1 = None
        if precision == 0 and recall == 0:
            F1 = 0
        else:
            F1 = 2*(precision*recall)/(precision + recall) 
        print("F1: ", F1)
        print("Total occurences: ", sum(list(metrics.values())))
        

In [49]:
print("Now evaluating dev data: ")
print()
results = evaluate(dev_x, dev_y)
print_evaluate(results)

Now evaluating dev data: 

Stats for IN:
Precision:  0.9944868532654793
Recall:  0.996600084997875
F1:  0.9955423476968795
dict_values([2345, 13, 8])
Total occurences:  2366
Stats for DT:
Precision:  0.9989690721649485
Recall:  0.9974266598044261
F1:  0.9981972701519444
dict_values([1938, 2, 5])
Total occurences:  1945
Stats for NNP:
Precision:  0.975268817204301
Recall:  0.9994490358126722
F1:  0.9872108843537416
dict_values([1814, 46, 1])
Total occurences:  1861
Stats for VBZ:
Precision:  0.9713831478537361
Recall:  0.953198127925117
F1:  0.9622047244094489
dict_values([611, 18, 30])
Total occurences:  659
Stats for NN:
Precision:  0.9244704570791528
Recall:  0.9928165219994014
F1:  0.9574253138981095
dict_values([3317, 271, 24])
Total occurences:  3612
Stats for ::
Precision:  0.9696969696969697
Recall:  0.9056603773584906
F1:  0.9365853658536586
dict_values([96, 3, 10])
Total occurences:  109
Stats for VBD:
Precision:  0.8804780876494024
Recall:  0.85
F1:  0.8649706457925636
dict_v

In [50]:
print("Now evaluating test data: ")
print()
results = evaluate(test_x, test_y)
print_evaluate(results)

Now evaluating test data: 

Stats for PRON:
Precision:  0.0
Recall:  0
F1:  0
dict_values([0, 2093, 0])
Total occurences:  2093
Stats for WP:
Precision:  0
Recall:  0.0
F1:  0
dict_values([0, 0, 93])
Total occurences:  93
Stats for X:
Precision:  0.0
Recall:  0
F1:  0
dict_values([0, 10827, 0])
Total occurences:  10827
Stats for IN:
Precision:  0
Recall:  0.0
F1:  0
dict_values([0, 0, 2314])
Total occurences:  2314
Stats for NNP:
Precision:  0
Recall:  0.0
F1:  0
dict_values([0, 0, 1998])
Total occurences:  1998
Stats for VBD:
Precision:  0
Recall:  0.0
F1:  0
dict_values([0, 0, 529])
Total occurences:  529
Stats for .:
Precision:  0
Recall:  0.0
F1:  0
dict_values([0, 0, 1451])
Total occurences:  1451
Stats for SCONJ:
Precision:  0.0
Recall:  0
F1:  0
dict_values([0, 237, 0])
Total occurences:  237
Stats for PROPN:
Precision:  0.0
Recall:  0
F1:  0
dict_values([0, 779, 0])
Total occurences:  779
Stats for ADP:
Precision:  0.0
Recall:  0
F1:  0
dict_values([0, 1252, 0])
Total occurence